<a href="https://colab.research.google.com/github/Samraddhi-Gupta/vision-image-transformers/blob/main/fake_news_clasifier_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fake News Classifier Using LSTM


Dataset: https://www.kaggle.com/c/fake-news/data#

In [ ]:
import pandas as pd
from google.colab import drive
import csv

drive.mount('/content/drive')

In [ ]:
csv.field_size_limit(1000000)  # You can set this to a higher value if needed
df=pd.read_csv('/content/drive/MyDrive/Temp/train.csv', on_bad_lines='warn', engine = 'python')
# df=pd.read_csv('train.csv', error_bad_lines=False, engine = 'python')

In [ ]:
df

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna()

In [ ]:
df.head()

In [ ]:
 ## Get the Independent Features

X = df.drop('label',axis=1)

In [ ]:
 ## Get the Independent Features

X = df.drop('label',axis=1)

In [ ]:
print(y.ndim)

In [ ]:
X.shape
X
print(X.dtypes)

In [ ]:
y.shape

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

one hot representation


In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]
messages['author'][1]

In [ ]:
messages

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
messages

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
from pandas.core.reshape.merge import string
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
corpus[5]

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

In [ ]:
corpus[1]

In [ ]:
onehot_repr[1]

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
embedded_docs=embedded_docs.astype(float)
print(embedded_docs)

In [ ]:
embedded_docs[1]

In [ ]:
embedded_docs[0]

In [ ]:
## Create a  model
embedding_vector_features=50 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(SimpleRNN(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
len(embedded_docs),y.shape

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
X_final=X_final.astype(float)

#y = np.asarray(y).astype("float64")
y_final=y
# y_final=y_final.astype(np.float)

In [ ]:
X_final.shape,y_final.shape,y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=42)
# for item in y_train:
#   print(item)
# len(y_train)
# for i in range(0,len(y_train)+1):
#   temp=y_train[i]
#   temp=int(temp)
#   y_train=temp

training of model

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=42)
# for item in y_train:
#   print(item)
# len(y_train)
# for i in range(0,len(y_train)+1):
#   temp=y_train[i]
#   temp=int(temp)
#   y_train=temp

adding dropout


In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Performance Metrics And Accuracy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [ ]:

from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))